In [ ]:
################### STEP 1: EDA ####################

In [47]:
import pandas as pd
n_rows=30000
data = pd.read_csv('train.csv', nrows=n_rows)
data.head()
# drop 'click', 'id', 'hour', 'device_id', 'device_ip' cloumns
df =data.drop(['click', 'id', 'hour', 'device_id', 'device_ip'], axis=1)
df.shape

(30000, 19)

In [48]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown='ignore')
X_encoded = enc.fit_transform(df)
X_encoded.shape

(30000, 4075)

In [49]:
y = data['click'].values
data['click'].value_counts()

0    24724
1     5276
Name: click, dtype: int64

In [50]:
# we need to split the data into training and testing sets. 
# Normally, we do so by randomly picking samples. 
# However, in our case, samples are in chronological order as indicated in the hour field. 
# Obviously, we cannot use future samples to predict the past ones
n_train =int(n_rows *0.9)
X_train = X_encoded[:n_train]
X_test = X_encoded[n_train:]
y_train = y[:n_train]
y_test = y[n_train:]

In [51]:
from sklearn.tree import DecisionTreeClassifier
parameters = {'max_depth':[3,10,None]}
decision_tree =DecisionTreeClassifier(criterion='gini', min_samples_split=30)
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(decision_tree, parameters, n_jobs=-1, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'max_depth': 10}


In [58]:
decision_tree_best =grid_search.best_estimator_
pos_prob = decision_tree_best.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_auc_score
print('The ROC AUC on testing set is:{0:.3f}'.format(roc_auc_score(y_test, pos_prob)))

The ROC AUC on testing set is:0.679


In [ ]:
############# Rondem forest ####################

In [63]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, criterion='gini',min_samples_split=30, n_jobs=-1)
grid_search = GridSearchCV(random_forest, parameters,n_jobs=-1, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'max_depth': None}


In [64]:
random_forest_best = grid_search.best_estimator_
pos_prob = random_forest_best.predict_proba(X_test)[:, 1]
print('The ROC AUC on testing set is:{0:.3f}'.format(roc_auc_score(y_test, pos_prob)))

The ROC AUC on testing set is:0.706
